<a href="https://colab.research.google.com/github/Bhar8at/Modified-Micrograd-for-Image-classification/blob/main/Neuralnetworkfromscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math, numpy as np, random

In [2]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = tf.keras.utils.normalize(x_train,1)
x_test = tf.keras.utils.normalize(x_test,1)
x_train_flattened = tf.reshape(x_train, (60000, 784))
x_test_flattened = tf.reshape(x_test, (10000, 784))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
tf.shape(x_train_flattened)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([60000,   784], dtype=int32)>

In [4]:
# One-hot encode the labels
y_train = tf.one_hot(y_train, depth=10)  # Converts to one-hot format
y_test = tf.one_hot(y_test, depth=10)

In [5]:
tf.shape(y_train)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([60000,    10], dtype=int32)>

In [6]:
import math
class Value:

    def __init__(self, data, _children = (), _op = ()):
        self.data = data
        self._op = _op
        self._prev = set(_children)
        self.grad = 0.0
        self._backward = lambda : None

    def __repr__(self):
        return f"Value(data={self.data})"

    def __pow__(self, other):
        assert isinstance(other, (int,float))
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad

        out._backward = _backward
        return out

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self,other), '+')

        def _backward():
           self.grad += 1.0 * out.grad
           other.grad += 1.0 * out.grad


        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data*other.data, (self,other), '*')

        def _backward():
           self.grad += other.data * out.grad
           other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self*other

    def __radd__(self, other):
        return self+other

    def __truediv__(self, other):
        return self * other**-1

    def tanh(self):
         x = self.data
         t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
         out = Value(t, (self,), 'tanh')

         def _backward():
           self.grad += (1 - t**2) * out.grad

         out._backward = _backward
         return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data *  out.grad

        out._backward = _backward

        return out

    def log(self):
        x = self.data
        out = Value(math.log(x), (self,), 'log')

        def _backward():
            self.grad += (1/self.data) * out.grad

        out._backward = _backward
        return out


    def backward(self):

        topo = []
        visited = []
        def build_topo(v):
            if v not in visited:
                visited.append(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [10]:
Value(1) * 1

Value(data=1)

In [22]:
x = Value(5)
x = x+7
x

Value(data=12)

In [23]:
Value(x)

Value(data=Value(data=12))

In [24]:
x = Value(x)
x._prev

set()

In [27]:

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return act

    def parameters(self):
        return self.w + [self.b]

class Layer():

    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for i in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP():

    def __init__(self, nin, nouts):
        s = [nin, ] + nouts
        self.layers = [Layer(s[i], s[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        x = [Value(xi) if not isinstance(xi, Value) else xi for xi in x]
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for Layer in self.layers for p in Layer.parameters()]

In [25]:
def softmax(logits):
    # Convert logits to Values if they aren't already
    logits = [Value(x) if not isinstance(x, Value) else x for x in logits]
    # Subtract max for numerical stability
    max_logit = max(x.data for x in logits)
    exp_logits = [l.data - max_logit for l in logits]
    exp_logits = [Value(math.exp(x)) for x in exp_logits]
    sum_exp = sum(e.data for e in exp_logits)
    return [e.data/sum_exp for e in exp_logits]

In [17]:
# class MLP:
#     def __init__(self, nin, nouts):
#         sizes = [nin] + nouts
#         self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(nouts))]

#     def __call__(self, x):
#         print("MLP is progress!")
#         for layer in self.layers:
#             print("Layer is progress!")
#             x = layer(x)  # Allow x to be a matrix (batch processing)
#         return x
#     def parameters(self):
#         return [p for layer in self.layers for p in layer.parameters()]

# class Layer:
#     def __init__(self, nin, nout):
#         self.w = np.random.uniform(-1, 1, (nout, nin))
#         self.b = np.random.uniform(-1, 1, nout)

#     def __call__(self, x):
#       print("Softmax is progress!")
#       return softmax(np.dot(Value(x), self.w.T) + self.b)

#     def parameters(self):
#         return [self.w, self.b]


In [28]:
nn = MLP(784, [10,10])

In [20]:
x = x_train_flattened.numpy()
x = x[:100]
y = y_train[:100]

In [29]:
nn(x[0])

TypeError: 'Value' object is not subscriptable

In [26]:
def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-7
    y_pred = [max(min(p, 1-epsilon), epsilon) for p in y_pred]
    return -sum(yt * math.log(yp) for yt, yp in zip(y_true, y_pred))

In [ ]:
for i in range(200):

    # forward pass
    yout = [nn(x[i]) for i in range(len(x))]
    yout = [softmax(yo) for yo in yout]
    loss = sum([cross_entropy_loss(y_true, y_pred) for y_true, y_pred in zip(y, yout)])

    # backward pass
    for p in nn.parameters():
        p.grad = 0
    loss.backward()

    # update
    for p in nn.parameters():
        p.data += -0.05*p.grad


    print("Step ",i,loss.data,)